In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#MNIST class prediction

mnist는 숫자를 손으로 그린 그림에 대한 데이터  
각 그림을 보고 어떠한 숫자가 적혀있는지 classification 하는 딥러닝 모델 구축  
multi class classification은 강의자료의 appendix 참조  

주요 모듈 목록  
**torch.utils.data.Dataset:** 데이터셋을 getitem method에서 한개씩 텐서형태로 반환하도록 구성하는 class  
**torch.utils.data.DataLoader:** batch processing을 위해 dataset에서 반환되는 데이터를 n개의 batch_size만큼 묶어서 반환하도록 하는 class  
**torch.nn.Module:** 딥러닝 모델을 구현한 class, forward method를 통해 입력된 텐서 데이터를 딥러닝 연산하여 결과 반환  
**torch.nn.CrossEntropyLoss:** loss 연산을 위한 class이며 backward method를 통해 gradient계산  
**torch.optim.Adam:** 계산된 gradient를 update해주어 feadient descent를 진행하는 class  

Dataset class구현  
  
dataset(fashion-mnist) : https://www.kaggle.com/datasets/zalando-research/fashionmnist?resource=download

traget: label(적힌 숫자)  
input feature: pixel1 ~ pixel 784까지의 명도
train/test file이 구분되어있음  

각 data를 torch.utils.data.Dataset을 통해 하나씩 load할 수 있는 class구현  
이후 torch.utils.data.DataLoader를 통해 batch개씩 변환

torch.utils.data.Dataset의 주요 method 
__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(데이터셋, 데이터경로)등을 생성  
__ len __: 해당 클래스에서 다루는 dataset의 길이를 반환하는 함수  
__ getitem __(index): index에 해당하는 데이터 하나를 tensor형태로 반환하는 함수

In [10]:
from sklearn.datasets import load_boston
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

from sklearn.preprocessing import MinMaxScaler


data_path = "/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/2일차_배포/dataset/HousingData.csv"
batch_size = 3

#torch.utils.data.Dataset을 상속하여 Dataset class선언
class myDataset(Dataset):
  #오브젝트를 선언할때 불러오는 함수, superclass(부모클래스)의 init을 실행해 주어야함
  def __init__(self, df_data) -> None:
    super().__init__()
    self.df_data = df_data #self를 사용하면 class내부에서 __init__ method만이 아닌 다른 method에서도 사용가능
    self.data_y = df_data.loc[:,["MEDV"]]
    self.data_x = df_data.drop(["MEDV"], axis=1)
      
  #list 형태의 class를 만들때 필수로 사용되는 함수, 전체 길이를 알아야 인덱싱이 가능
  def __len__(self):
    return len(self.data_y)

  #index에 해당하는 데이터를 반환해주는 함수
  def __getitem__(self, index):
    data = torch.Tensor(self.data_x.loc[index,:])
    target = torch.Tensor(self.data_y.loc[index,:])

    return data, target

#data load후 train(400개)/test(나머지)데이터를 분할
data_df = pd.read_csv(data_path).dropna()
train_data_df = data_df.loc[:400,:].reset_index()
test_data_df = data_df.loc[400:,:].reset_index()

#각 dataset을 선언
trainDataset = myDataset(train_data_df)
testDataset = myDataset(test_data_df)

#선언된 dataset을 dataloader를 통해 batch processing
trainDataloader = DataLoader(trainDataset, batch_size = batch_size)
testDataloader = DataLoader(testDataset, batch_size = batch_size)

#잘 작동하는지 test
for i in trainDataset:
  print("dataset test")
  print(i)
  break

#잘 작동하는지 test
for i in trainDataloader:
  print("data loader test")
  data = i[0]
  target = i[1]
  print(data)
  print(data.shape)
  print(target)
  print(target.shape)
  break


dataset test
(tensor([0.0000e+00, 6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
        6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01,
        3.9690e+02, 4.9800e+00]), tensor([24.]))
data loader test
tensor([[0.0000e+00, 6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
         6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01,
         3.9690e+02, 4.9800e+00],
        [1.0000e+00, 2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9690e+02, 9.1400e+00],
        [2.0000e+00, 2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9283e+02, 4.0300e+00]])
torch.Size([3, 14])
tensor([[24.0000],
        [21.6000],
        [34.7000]])
torch.Size([3, 1])


Deep learning 모델 구현  
10개의 label의 확률을 예측해야하는 모델  
그렇기에 output layer의 최종 결과가 10차원의 vector가 되어야함  
torch.nn.Module을 이용하여 모델 구현  
1st hidden layer의 feature는 10개  
2nd hidden layer의 feature는 100개  
인 모델을 구현한다.  

torch.nn.Linear: perceptron의 weighted sum과 같이 linaer regression연산을 하는 calss  
torch.nn.ReLU: ReLU activation function을 수행하는 class  

torch.nn.Module의 주요 method  
__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(사용할 deep learning layer, activation function, 등)등을 생성  
__ forward __(data): 입력받은 data를 딥러닝 모델을 통해 결과를 예측하여 반환하는 class  

**각 딥러닝 레이어 연산 중 차원수를 확인하고 잘 맞춰줄 것**  
참고 document(해당 사이트의 shape를 확인하고 tensor형태 결정)  
nn.Linear: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html  
nn.ReLU: https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

In [ ]:
from torch import nn

#딥러닝 모델을 작성하기위한 모듈
class myModel(nn.Module):
  #오브젝트를 선언할 때 불러와지는 함수 일반적으로 이번 모델에서 사용될 각 레이어들이 포함됨
  def __init__(self) -> None:
      super().__init__()

      #1st_hidden: 10, 2nd_hidden: 100, output: 10의 형태에 맞는 linear layer들을 선언, activation으로 Relu사용
    
  #데이터를 입력받고 딥러닝 연산후 결과를 반환하는 함수
  def forward(self, x):

#작성한 모델 선언
model = myModel()


작성한 dataset과 model을 이용하여 딥러닝 프로세스 구현  

pytorch 딥러닝 프로세스
1. dataset, model선언
2. dataset과 model을 통한 결과 예측
3. 예측된 결과를 통해 **loss**연산 및 **loss.backward**
4. **optimizer.step()**를 사용하여 graident update

주요 오브젝트  
torch.nn.CrossEntropyLoss: 문장 분류를 위한 loss 수행  
torch.optim.Adam: Adam optimizer를 통해 gradient update를 수행하는 class

각 오브젝트의 입력과 선언은 다음 doc 참조:  
CrossEntropyLoss: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html  
Adam: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html

In [ ]:
from torch.optim import Adam
from torch.nn import MSELoss

#학습을 위한 optimizer와 loss function 설정

#100번의 에폭을 실행
for e in range(100):
  #train
  #선언한 모델 오브젝트를 학습가능한 상태로 변경

  #모든 학습데이터에 대해서 학습
  for i in trainDataloader:
    #매 배치에 대한 gradient계산 이전에 optimizer에 저장된 이전 batch에 gradient를 삭제(초기화)

    #데이터 분리

    #결과 도출

    #loss연산

    #loss backpropagation

    #gradient update



  #test
  #model이 학습되지 않는 상태로 변경

  #gradient를 계산하지 않도록 하여 cost낭비 방지

    #모든 test dataset에 대해서 결과연산

    
